In [5]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import csv


# Get the list of NASDAQ companies
nasdaq_tickers = pd.read_csv('nasdaq_screener.csv')
#nasdaq_tickers= pd.read_csv('https://datahub.io/core/nasdaq-listings/r/nasdaq-listed-symbols.csv')
tickers_total = nasdaq_tickers['Symbol'].tolist()
tickers = tickers_total[10:13]

# Prepare the CSV file
header_written = False
csv_file = 'nasdaq_financial_data.csv'


# Helper function to get the stock price on a specific date
def get_stock_price(ticker, date):
    for i in range(7):        
        try:
            adjusted_date = date + timedelta(days=i)
            stock_data = yf.download(ticker, start=adjusted_date, end=adjusted_date + timedelta(days=1), progress=False)
            if not stock_data.empty:
                return stock_data['Close'].iloc[0]
        except Exception as e: 
            print(f"Error processing ticker {ticker}: {e}")



# Function that flattens the data of a dataframe concatenate the columns and put everything in a row
def flatten_data(df, ticker):    
    
    df_u = df.unstack().reset_index()
    df_u['Ticker'] = df_u.apply(lambda row: str(row['level_0'].year) + ' ' + row['level_1'], axis=1)
    df_u.set_index(keys = 'Ticker')
    df_ordered = df_u.drop(columns = ['level_0','level_1'])
    df_transposed = df_ordered.set_index(keys='Ticker').T
    df_transposed['Ticker'] = ticker
    df_transposed.set_index(keys='Ticker',inplace = True)  
    
    return df_transposed




In [6]:
#Compute the stock value ratio (item_stock%/sp500_stock%) in a year

def get_stock_value_ratio(date, ticker):
    for i in range(7):
        print(i)
        try:
            # Adjust date by i days
            adjusted_date = date + timedelta(days=i)
            stock_data = get_stock_price(ticker, adjusted_date)
            
            if not stock_data.empty:
                date_diff = adjusted_date - timedelta(weeks=52)
                #change in s&p 500 price in %     
                sp500_change = get_stock_price("^GSPC", adjusted_date) / get_stock_price("^GSPC",date_diff)
                #change in stock price in %
                stock_change = get_stock_price(ticker,adjusted_date)/ get_stock_price(ticker,date_diff)
                #ratio of change_stock / change_s&p500
                ratio = stock_change / sp500_change
                if ratio is not None: 
                    return ratio
        except Exception as e: 
            print(f"Error processing ticker {ticker}: {e}")

In [7]:
#function to get the stock value corresponding to the date date

def get_stock_value(df, ticker) :
    lst_stock_value = []
    lst_stock_ratio = []
    for date in df.columns :
        date_str = date.strftime('%Y-%m-%d')
        lst_stock_value.append(get_stock_price(ticker, date))
        lst_stock_ratio.append(get_stock_value_ratio(ticker, date))
        
    df.loc['Stock value'] = lst_stock_value
    df.loc['Ratio ticker sp500'] = lst_stock_ratio
    return df

In [8]:
def create_df(tickers):
    df_complete = pd.DataFrame()
    for ticker in tickers:  
        df_row = pd.DataFrame()
        try: 
            # Get financial statements for the ticker ticker
            stock = yf.Ticker(ticker)
            balance_sheet = stock.balance_sheet
            income_statement = stock.financials
            cashflow_statement = stock.cashflow
            
            # add the stock value and flatten the dataframe
            df_bs = get_stock_value(balance_sheet,ticker)
            df_bs_f = flatten_data(df_bs,ticker)
            df_is_f = flatten_data(income_statement,ticker)
            df_cs_f = flatten_data(cashflow_statement,ticker)
            #concatenate
            df_row = pd.concat([df_bs_f,df_is_f,df_cs_f],axis=1)
            df_complete = pd.concat([df_complete, df_row])
    
        except Exception as e: 
            print(f"Error processing ticker {ticker}: {e}")
    return df_complete
        